# Supercrunchers

## Imports

In [127]:
%matplotlib inline

In [128]:
import pandas as pd
import numpy as np
import itertools

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LogisticRegression
from sklearn.svm import SVR, SVC
from sklearn.model_selection import train_test_split, KFold, cross_validate, StratifiedKFold
from sklearn.metrics import r2_score, mean_squared_error, get_scorer_names, classification_report
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import FunctionTransformer
from sklearn.utils import resample
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from category_encoders.target_encoder import TargetEncoder

In [129]:
pd.set_option("display.precision", 2)

## Data Loading and Inspection

In [130]:
# Get the dataset
file_path = 'data/soccer_data.dta'
csv_path = 'data/soccer_data.csv'
df_all = pd.read_stata(file_path)
df_all.to_csv(csv_path)
df_all['date'] = pd.to_datetime(df_all['date'], format='%d/%m/%Y')

In [131]:
df_all.head()

,v1,competition,competition_id,date,match,match_id,team,team_id,player,player_id,...,position,position_role,rat,kicker,bild,skysports,goalkeeper,defender,midfielder,forward
0,0,Euro 2016,102,2016-06-10,"France - Romania, 2 - 1",1694390,Romania,11944,Dragos Grigore,84536,...,Defender,DC,Kicker,1,0,0,0,1,0,0
1,1,Euro 2016,102,2016-06-10,"France - Romania, 2 - 1",1694390,Romania,11944,Dragos Grigore,84536,...,Defender,DC,WhoScored,0,0,0,0,1,0,0
2,2,Euro 2016,102,2016-06-10,"France - Romania, 2 - 1",1694390,Romania,11944,Dragos Grigore,84536,...,Defender,DC,SofaScore,0,0,0,0,1,0,0
3,4,Euro 2016,102,2016-06-10,"France - Romania, 2 - 1",1694390,Romania,11944,Mihai Pintilii,83824,...,Midfielder,DMC,Kicker,1,0,0,0,0,1,0
4,5,Euro 2016,102,2016-06-10,"France - Romania, 2 - 1",1694390,Romania,11944,Mihai Pintilii,83824,...,Midfielder,DMC,WhoScored,0,0,0,0,0,1,0


In [132]:
# def get_match_result(match:str, is_home_team:bool):
#     score = match[-5:]
#     score = score.split(" - ")
#     result = 0
#     if score[0] == score[1]:
#         result = 0

#     if score[0] > score[1]:
#         result = 1

#     if score[0] < score[1]:
#         result = -1

#     if not is_home_team:
#         result = -1 * result

#     return result

# # df_all[df_all['rat'] == 'SofaScore'][['match', 'is_home_team']].apply(lambda x: get_match_result(x['match'], x['is_home_team']), axis=1)
# # df_clean['result'] = df[['match', 'is_home_team']].apply(lambda x: get_match_result(x['match'], x['is_home_team']), axis=1)


In [133]:
# df_all[['player_id', 'match_id']].drop_duplicates()
df_all[['player']].value_counts()
# df_all.loc[(df_all['player_id'] == 84536) & (df_all['rat'] == 'WhoScored'), ['original_rating', 'date']]
# df_all[['rat', 'is_human', 'original_rating']].describe()
# df_all.groupby('rat')['original_rating'].describe()
# df_all['position'].unique()

player          
Benjamin Pavard     119
Yann Sommer         119
Thibaut Courtois    117
Kevin De Bruyne     116
Hugo Lloris         115
                   ... 
Damien Delaney        2
Paulo Gazzaniga       2
Michel Vorm           2
Lasse Srrensen        1
Beni Baningime        1
Length: 1445, dtype: int64

## Preprocessing

In [134]:
def filter_columns(df, filters):
    for col, val in filters.items():
        df = df.loc[df[col] == val]

    return df

def clean_columns(df):
    # drop some columns
    useless_columns = ['v1', 'rating', 'team_rating', 'kicker', 'bild', 'skysports', 'goalkeeper', 'defender', 'midfielder', 'forward']
    redundant_columns = ['competition_id', 'match_id', 'team_id', 'player', 'win', 'lost', 'position']
    
    nonfeature_columns = ['player_id', 'team_pos_rating', 'team_rating_original', 'past_performances', 
    'opp_rating', 'opp_rating_original', 'opp_gk_rating', 'opp_bestdf_rating', 'opp_bestmf_rating', 'opp_bestfw_rating', 
    'rat', 'is_human']

    different_encoded_columns = ['match', 'date']
    df_clean = df.drop(columns=useless_columns + redundant_columns + nonfeature_columns + different_encoded_columns)

    def get_match_result(match:str, is_home_team:bool):
        score = match[-5:]
        score = score.split(" - ")
        result = 0
        if score[0] == score[1]:
            result = 0

        if score[0] > score[1]:
            result = 1

        if score[0] < score[1]:
            result = -1

        if not is_home_team:
            result = -1 * result

        return result

    # add columns
    df_clean['result'] = df[['match', 'is_home_team']].apply(lambda x: get_match_result(x['match'], x['is_home_team']), axis=1)

    # date
    date = pd.to_datetime(df['date'], format="%d/%m/%Y")
    df_clean['weekday'] = date.dt.weekday
    df_clean['month'] = date.dt.month
    return df_clean


def split_dataset(df):
    # Split the df into X and y
    X = df.drop(columns=['original_rating'])
    y = df['original_rating']

    return X, y

In [135]:
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))


def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))


def create_pipeline(df, model):
    # define feature types
    numeric_features = set(df.select_dtypes(
        exclude=["category", "object"]).columns)
    categorical_features = set(df.select_dtypes(
        include=['category', "object"]).columns)
    cyclic_features = {'weekday', 'month'}
    team_feature = {'team'}
    
    numeric_features -= cyclic_features
    categorical_features -= cyclic_features
    categorical_features -= team_feature

    numeric_features = list(numeric_features)
    categorical_features = list(categorical_features)
    cyclic_features = list(cyclic_features)
    team_feature = list(team_feature)

    # print(numeric_features)
    # print(categorical_features)
    # print(cyclic_features)

    # transformer for numeric features
    numeric_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(missing_values=np.nan, strategy='median')),
            # ("scaler", RobustScaler()),
            # ("scaler", StandardScaler()),
        ]
    )

    # transformer for categorical features
    categorical_transformer = Pipeline(
        steps=[
            # ("imputer", SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
            ("ohe", OneHotEncoder(handle_unknown='ignore', sparse=False)),
        ]
    )

    # preprocessing transformer, applies different transformations on different features
    preprocessor = ColumnTransformer(
        transformers=[
            ("numeric", numeric_transformer, numeric_features),
            ("categorical", categorical_transformer, categorical_features),
            ("team", TargetEncoder(handle_missing='value', handle_unknown='value'), team_feature),
            ("month_sin", sin_transformer(12), ["month"]),
            ("month_cos", cos_transformer(12), ["month"]),
            ("weekday_sin", sin_transformer(7), ["weekday"]),
            ("weekday_cos", cos_transformer(7), ["weekday"]),
        ]
    )

    # final pipeline preprocessing + classifier
    pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("scaler", StandardScaler()),
            # ('pca', PCA(n_components='mle')),
            ("classifier", model),
        ]
    )

    return pipe


In [136]:
# To see the transformed dataset that the model will actually run on
df_f = filter_columns(df_all, {'rat': 'Kicker', 'position': 'Forward'})
df_temp = clean_columns(df_f)
X_temp, y_temp = split_dataset(df_temp)
preprocessor = create_pipeline(X_temp, None)
preprocessor.steps.pop(-1)
temp = preprocessor.fit_transform(X_temp, y_temp)
temp = pd.DataFrame(temp)
temp.describe()

,0,1,2,3,4,5,6,7,8,9,...,95,96,97,98,99,100,101,102,103,104
count,1403.0,1.40e+03,1.40e+03,1.40e+03,1.40e+03,1.40e+03,1.40e+03,1.40e+03,1.40e+03,1.40e+03,...,1.40e+03,1.40e+03,1.40e+03,1.40e+03,1.40e+03,1.40e+03,1.40e+03,1.40e+03,1.40e+03,1.40e+03
mean,0.0,-1.77e-17,-1.24e-16,-1.27e-17,-6.33e-18,1.09e-16,7.34e-17,-3.80e-17,1.01e-17,-7.98e-17,...,-1.01e-16,1.62e-16,-1.27e-17,-4.30e-17,2.03e-17,-1.33e-14,1.62e-16,8.10e-17,-7.31e-15,1.33e-15
std,0.0,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,...,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00,1.00e+00
min,0.0,-2.67e-02,-1.53e+00,-2.67e-02,-5.15e-01,-1.54e+00,-1.68e+00,-8.85e-01,-1.85e+00,-8.09e-01,...,-3.56e-01,-3.87e-01,-1.89e+00,-3.52e-01,-3.50e-01,-2.10e+00,-1.52e+00,-1.28e+00,-6.35e-01,-1.64e+00
25%,0.0,-2.67e-02,-6.54e-01,-2.67e-02,-5.15e-01,-6.49e-01,-7.53e-01,-8.85e-01,-7.23e-01,-8.09e-01,...,-3.56e-01,-3.87e-01,5.30e-01,-3.52e-01,-3.50e-01,-5.52e-01,-7.84e-01,-1.09e+00,-6.35e-01,-3.68e-01
50%,0.0,-2.67e-02,-1.71e-01,-2.67e-02,-5.15e-01,4.91e-02,-1.38e-01,-4.40e-01,-9.24e-02,2.03e-01,...,-3.56e-01,-3.87e-01,5.30e-01,-3.52e-01,-3.50e-01,-8.15e-02,-4.95e-02,9.66e-02,-6.35e-01,-3.68e-01
75%,0.0,-2.67e-02,4.73e-01,-2.67e-02,6.78e-01,6.36e-01,5.80e-01,4.51e-01,6.48e-01,2.03e-01,...,-3.56e-01,-3.87e-01,5.30e-01,-3.52e-01,-3.50e-01,5.91e-01,1.22e+00,7.83e-01,2.66e-01,1.21e+00
max,0.0,3.74e+01,8.17e+00,3.74e+01,6.64e+00,6.21e+00,3.76e+00,1.11e+01,4.70e+00,6.28e+00,...,2.81e+00,2.58e+00,5.30e-01,2.84e+00,2.86e+00,2.92e+00,1.42e+00,1.47e+00,2.61e+00,1.92e+00


## Group Comparison Testing

In [137]:
def test_group_effects(df_base: pd.DataFrame):
    # clean the df for classification task
    df = clean_columns(df_base)
    df: pd.DataFrame = df.drop(columns=['original_rating'])
    df = df.drop_duplicates()
    X = df

    # get the target value and encode it
    y = df_base['player'].iloc[df.index]
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    models = [RandomForestClassifier(min_samples_leaf=10), DummyClassifier(strategy='stratified', random_state=42)]
    for model in models:
        pipe = create_pipeline(X, model)

        # do cross validation
        s = cross_validate(
            pipe,
            X,
            y,
            cv=StratifiedKFold(10),
            scoring=['accuracy', 'f1_micro', 'f1_macro'],
            n_jobs=-1,
        )
        # evaluate

        acc_scores = s['test_accuracy']
        f1_micro_scores = s['test_f1_micro']
        f1_macro_scores = s['test_f1_macro']

        acc = np.percentile(acc_scores, 100)
        f1_micro = np.percentile(f1_micro_scores, 100)
        f1_macro = np.percentile(f1_macro_scores, 100)

        print({'model': model.__class__.__name__,'acc': acc, 'f1_micro':f1_micro, 'f1_macro': f1_macro})



test_group_effects(df_all)

/Users/ahmetayrnc/Library/Python/3.8/lib/python/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


{'model': 'RandomForestClassifier', 'acc': 0.3269118505545826, 'f1_micro': 0.3269118505545826, 'f1_macro': 0.18871516483590786}


/Users/ahmetayrnc/Library/Python/3.8/lib/python/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


{'model': 'DummyClassifier', 'acc': 0.002335084646818447, 'f1_micro': 0.002335084646818447, 'f1_macro': 0.0013175230566534913}


## Bootstrapping

In [138]:
class Bootstrap:
    def __init__(self, nr):
        self.nr = nr
    
    def split(self, X, y, groups=None):
        idx = range(len(X))
        splits = []
        for i in range(self.nr):
            train = resample(idx, replace=True, n_samples=len(X), random_state=i)
            test = list(set(idx) - set(train))
            splits.append((train, test))
        return splits


class GroupedBootstrap:
    def __init__(self, nr):
        self.nr = nr
    
    def split(self, X, y, groups):
        idx = X.index
        splits = []
        for i in range(self.nr):
            # sample the players
            train_players =  resample(groups.unique(), replace=True, random_state=i)

            # collect all the records for the players in our player samples
            train = []
            for train_player in train_players:
                train_subset = groups.loc[groups == train_player]
                train += train_subset.index.tolist()

            # remove the sampled records from all records to get the out of bag records
            test = list(set(idx) - set(train))

            # append the split to the splits
            splits.append((train, test))

        return splits

def test_grouped_bs(df):
    # create datasets
    filter_dict = {'rat':'Kicker'}
    df = filter_columns(df, filter_dict)
    groups = df['player']
    df = clean_columns(df)
    X, y = split_dataset(df)

    gbs = GroupedBootstrap(10)
    splits = gbs.split(X, y, groups)
    # print(len(splits))
    # print(len(splits[0][0]))
    # print(len(splits[0][1]))


test_grouped_bs(df_all)

## Hyper-parameter Optimizations

## Evaluation

In [139]:
def train_and_test(df):
    groups = df['player']
    # create datasets
    df = clean_columns(df)
    X, y = split_dataset(df)

    # create the pipeline
    model = Ridge()
    model = RandomForestRegressor(min_samples_leaf=20, n_jobs=-1, random_state=42, verbose=0)
    pipe = create_pipeline(X, model)

    # do cross validation
    s = cross_validate(
        pipe,
        X,
        y,
        cv=Bootstrap(100),
        scoring=['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error'],
        n_jobs=-1,
        groups=groups,
    )

    # evaluate
    r2_scores = s['test_r2']
    mae_scores = s['test_neg_mean_absolute_error'] * -1
    mse_scores = s['test_neg_mean_squared_error'] * -1

    r2 = np.percentile(r2_scores, 95)
    mae = np.percentile(mae_scores, 95)

    # sigma2
    mse = np.percentile(mse_scores, 95)
    sigma2 = mse * (X.shape[0] / (X.shape[0] - X.shape[1] + 1))
    
    return {'r2': r2, 'mae': mae, 'sigma^2': sigma2}


def evaluate(df):
    ratings = ['Kicker', 'WhoScored', 'SkySports', 'Bild', 'SofaScore', 'The Guardian']
    positions = ['Forward', "Midfielder", 'Defender', 'Goalkeeper']
    results = []

    for rat, pos in itertools.product(ratings, positions):
        filter_dict = {'rat':rat, 'position': pos}
        df_eval = filter_columns(df, filter_dict)
        result = train_and_test(df_eval)
        res = {**filter_dict, **result}
        results.append(res)
        print(res)

    return pd.DataFrame(results)

results_df = evaluate(df_all)
results_df


{'rat': 'Kicker', 'position': 'Forward', 'r2': 0.6655114580545157, 'mae': 0.5452299184896905, 'sigma^2': 0.5336459775360425}
{'rat': 'Kicker', 'position': 'Midfielder', 'r2': 0.5044054336688301, 'mae': 0.5347814713812554, 'sigma^2': 0.4568740087682472}
{'rat': 'Kicker', 'position': 'Defender', 'r2': 0.41536543079230526, 'mae': 0.5350746309583487, 'sigma^2': 0.4664665629924159}
{'rat': 'Kicker', 'position': 'Goalkeeper', 'r2': 0.27621110733934373, 'mae': 0.5732850602366605, 'sigma^2': 0.6672685502361346}
{'rat': 'WhoScored', 'position': 'Forward', 'r2': 0.8702695354212715, 'mae': 0.26359358835380486, 'sigma^2': 0.12068008153852894}
{'rat': 'WhoScored', 'position': 'Midfielder', 'r2': 0.7760643459995463, 'mae': 0.27271773791154325, 'sigma^2': 0.12687294850631825}
{'rat': 'WhoScored', 'position': 'Defender', 'r2': 0.7029448111371758, 'mae': 0.2813630716357095, 'sigma^2': 0.1384641547477386}
{'rat': 'WhoScored', 'position': 'Goalkeeper', 'r2': 0.7772767799839359, 'mae': 0.30631664964433325

,rat,position,r2,mae,sigma^2
0,Kicker,Forward,0.67,0.55,0.53
1,Kicker,Midfielder,0.50,0.53,0.46
2,Kicker,Defender,0.42,0.54,0.47
3,Kicker,Goalkeeper,0.28,0.57,0.67
4,WhoScored,Forward,0.87,0.26,0.12
5,WhoScored,Midfielder,0.78,0.27,0.13
6,WhoScored,Defender,0.70,0.28,0.14
7,WhoScored,Goalkeeper,0.78,0.31,0.18
8,SkySports,Forward,0.54,0.68,0.80
9,SkySports,Midfielder,0.46,0.62,0.68


In [140]:
# r2: 0.522, mae: 0.521, sigma2: 0.437, error: 1.916
# r2: 0.766, mae: 0.280, sigma2: 0.132, error: 0.824
# r2: 0.450, mae: 0.602, sigma2: 0.588, error: 2.380